<a href="https://colab.research.google.com/github/Dmitri9149/TensorFlow-PyTorch-basics/blob/master/PyTorch_Padding_Stride_MultiChannels_Pooling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -U: Upgrade all packages to the newest available version
!pip install -U d2l

from d2l import torch as d2l
import torch as torch
from torch import nn
import numpy as np

     |████████████████████████████████| 61kB 4.3MB/s 


Exercises and some code modification/experimentation for the d2l.ai book : http://d2l.ai/ 

Padding

In [2]:
### initialize convolutional layer , make some changes in sizes of the layer 
### with the help of padding
def comp_conv2d(conv2d, X):
  ### (1,1) correspond to batch size and num of channels
  X = X.reshape((1,1) +  X.shape)
### elim dimentions we do not need

  Y = conv2d(X)
  return Y.reshape(Y.shape[2:])

conv2d = nn.Conv2d(1,1,kernel_size = 3, padding = 1)
X = torch.rand(size = (8,8))
comp_conv2d(conv2d,X).shape


torch.Size([8, 8])

In [3]:
conc2d = nn.Conv2d(1,1, kernel_size = (5,3), padding = [2,1])
comp_conv2d(conv2d,X).shape

torch.Size([8, 8])

Stride

In [4]:
conv2d = nn.Conv2d(1,1, kernel_size = 3, padding = 1, stride = 2)
comp_conv2d(conv2d,X).shape

torch.Size([4, 4])

In [5]:
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])

In [6]:
### input X and kernel K , compute cross correlation
def corr2d(X,K):
  reduce_sum = lambda x, *args, **kwargs: x.sum(*args, **kwargs)
  h,w = K.shape
  Y=torch.zeros(X.shape[0] -h + 1, X.shape[1] - w + 1)
  for i in range(Y.shape[0]):
    for j in range(Y.shape[1]):
      Y[i,j]=reduce_sum((X[i:i+ h,j:j+w])*K)
  return Y

MultiChannels

In [7]:
### work with multi channels
def corr2d_multi_in(X,K):
  ## iterate via the first dimention (channels); sum results together
  ## zip 'list' of 2D inputs and kernels 
  corr = (corr2d(x,k) for x, k in zip(X,K))
  return sum(corr)

In [8]:
X = torch.tensor([[[0.0, 3.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [1.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 55.,  68.],
        [ 98., 113.]])

In [9]:
### multiple output and input
### size of input is (i, h, w) size of kernel is (o,i,h_k,w_k) 
def corr2d_multi_in_out(X,K):
## iterate via o -> first dim of kernel and at every step make cross correlation 
### with input X
############# iterate over first dim of K 
  return torch.stack([corr2d_multi_in(X,k) for k in K ], dim = 0)

In [10]:
K = torch.stack((K,K+1,K+2,K+3))
K.shape

torch.Size([4, 2, 2, 2])

In [11]:
corr2d_multi_in_out(X,K)

tensor([[[ 55.,  68.],
         [ 98., 113.]],

        [[ 77.,  98.],
         [142., 165.]],

        [[ 99., 128.],
         [186., 217.]],

        [[121., 158.],
         [230., 269.]]])

1 x 1 Convolutional Layers

In [12]:
def corr2d_multi_in_out_1x1(X,K):
  i,h,w = X.shape
  o = K.shape[0]
  X = X.reshape((i,h*w))
  K = K.reshape((o,i))
##  mutmul in fully connected layer
  Y = torch.matmul(K,X)
  return Y.reshape((o,h,w))


In [13]:
## 2 input channels
X = torch.normal(0, 1, (2, 3, 3)) 
### kernel will make deal with 2 inp. channels and will generate 4 outp. channels
K = torch.normal(0, 1, (4 , 2, 1, 1))
Y1 = corr2d_multi_in_out_1x1(X,K)
Y2 = corr2d_multi_in_out(X, K)

In [14]:
reduce_sum = lambda x, *args, **kwargs: x.sum(*args, **kwargs)
assert float(reduce_sum(torch.abs(Y1-Y2))) <1e-6
### no assertion error

In [15]:
Y1

tensor([[[-0.0887, -0.3703, -1.5571],
         [-0.2064,  0.2859,  0.7987],
         [ 0.0425, -1.0824,  0.3186]],

        [[-0.4925,  2.1469, -0.4046],
         [-0.9914,  2.3095,  0.5052],
         [-3.6363, -0.1164, -0.7965]],

        [[-0.1735, -0.0382, -1.7009],
         [-0.3786,  0.6773,  0.9210],
         [-0.5492, -1.1554,  0.2045]],

        [[ 0.7012, -4.7697, -2.7834],
         [ 1.3484, -3.5821,  0.8826],
         [ 6.7552, -2.2366,  2.1798]]])

Pooling

In [17]:
def pool2d(X, pool_size, mode = 'max'):
  h,w = pool_size
## initialize Y tenzor 
  Y = torch.zeros((X.shape[0]-h+1,X.shape[1]-w+1))
  for i in range(Y.shape[0]):
    for j in range(Y.shape[1]):
      if mode == 'max':
        Y[i,j]=X[i:i+h,j:j+w].max()
      elif mode == 'avg':
        Y[i,j]=X[i:i+h,j:j+w].mean()
    return Y



In [18]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2))

tensor([[4., 5.],
        [0., 0.]])

In [19]:
pool2d(X, (2, 2), 'avg')

tensor([[2., 3.],
        [0., 0.]])

In [21]:
## generate 1d tensor, fill with increasing numbers , reshape to  4 dim input 
X = d2l.reshape(torch.arange(start = 0, end = 32, step = 2, dtype=torch.float32), (1, 1, 4, 4))
X

tensor([[[[ 0.,  2.,  4.,  6.],
          [ 8., 10., 12., 14.],
          [16., 18., 20., 22.],
          [24., 26., 28., 30.]]]])

In [22]:
## by default in MaxPool2d pooling layer stride = 1
pool2d = nn.MaxPool2d(3)
pool2d(X)

tensor([[[[20.]]]])

In [23]:
## stride and padding can be manually adjusted 
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)


tensor([[[[10., 14.],
          [26., 30.]]]])

In [24]:
pool2d = nn.MaxPool2d((2, 3), padding=(1, 1), stride=(2, 3))
pool2d(X)

tensor([[[[ 2.,  6.],
          [18., 22.],
          [26., 30.]]]])

Multiple channels : pooling pools each input channel separatelly, number of channels is not changing.

In [26]:
## cat 3 tensors within dim = 1, channel dimention ( dim = 0 -> batches)
Y = torch.cat((X,X-1,X+2),1)
Y

tensor([[[[ 0.,  2.,  4.,  6.],
          [ 8., 10., 12., 14.],
          [16., 18., 20., 22.],
          [24., 26., 28., 30.]],

         [[-1.,  1.,  3.,  5.],
          [ 7.,  9., 11., 13.],
          [15., 17., 19., 21.],
          [23., 25., 27., 29.]],

         [[ 2.,  4.,  6.,  8.],
          [10., 12., 14., 16.],
          [18., 20., 22., 24.],
          [26., 28., 30., 32.]]]])

In [27]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(Y)

tensor([[[[10., 14.],
          [26., 30.]],

         [[ 9., 13.],
          [25., 29.]],

         [[12., 16.],
          [28., 32.]]]])